In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, Kernel, Hyperparameter
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import GPy

In [ ]:
# Generate optimized LHS
lhs = qmc.LatinHypercube(d=3)
samples = lhs.random(n=24)

#plt.scatter(samples[:, 0], samples[:, 1]) # for 2d 

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(samples[:, 0], samples[:, 1], samples[:, 2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.title('lhs test')
plt.show()

# Print x, y, z values in a readable format
print("X, Y, Z values:")
for i, (x, y, z) in enumerate(samples):
    print(f"Point {i+1}: x={x:.4f}, y={y:.4f}, z={z:.4f}")

In [6]:
class CustomKernel(Kernel):
    def __init__(self, variance = 1.0):
        self.variance = variance # kernel variance
        #define variance as a hyperparameter (log-scaled for positive values)
        self.hyperparameter_variance = Hyperparameter("variance", "log", 1e-2, 10.0)
    def __call__(self, X, Y = None, eval_gradient = False):
        if Y is None:
            Y = X
        
        # Compute categorical overlap (1 if same, 0 if different)
        diff = X[:, None] != Y[None, :] # Boolean matrix (True if different)
        k_cat = self.variance * (1-np.mean(diff, axis=-1)) # normalizes overlap

        if eval_gradient:
            # Gradient w.r.t. variance
            grad = np.ones_like(k_cat)[..., np.newaxis] # Shape: (n_samples, n_samples, 1)
            return k_cat, grad
        return k_cat
    
    def diag(self, X):
        return np.full(X.shape[0], self.variance) # self-similarity
    
    def is_stationary(self):
        return False
    
class CategoryOverlapKernelGPy(GPy.kern.Kern):
    """GPy implementation of a categorical overlap kernel."""

    def __init__(self, input_dim, variance=1.0, active_dims=None, name='catoverlap'):
        super().__init__(input_dim, active_dims=active_dims, name=name)
        self.variance = GPy.core.parameterization.Param('variance', variance)
        self.link_parameter(self.variance)

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X

        diff = X[:, None] - X2[None, :]
        diff[np.where(np.abs(diff))] = 1  # Mark different categories
        k_cat = self.variance * (1 - np.mean(diff, axis=-1))  # Normalize overlap
        return k_cat



In [8]:
X_test = np.array([[0], [1], [2], [1], [0]]) #each row is a category

# Create kernel and compute the kernel matrix
gpy_kernel = CategoryOverlapKernelGPy(input_dim=1, variance=1.0)
K_gpy = gpy_kernel.K(X_test)
print("GPy Kernel Matrix:\n", K_gpy)

#create kernel and compute the kernel matrix
skl_kernel = CustomKernel(variance=1.0)
K_skl = skl_kernel(X_test)
print("SKL Kernel Matrix:\n", K_skl)

GPy Kernel Matrix:
 [[1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1.]]
SKL Kernel Matrix:
 [[1. 0. 0. 0. 1.]
 [0. 1. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1.]]


In [ ]:
class CustomKernel(Kernel):
    """Custom Kernel that adds a Matern Kernel with an extra exponential term."""
    
    def __init__(self, length_scale=1.0, custom_param=1.0):
        self.length_scale = length_scale
        self.custom_param = custom_param
    
    def __call__(self, X, Y=None, eval_gradient=False):
        """Compute the kernel matrix."""
        dists = np.linalg.norm(X[:, np.newaxis] - Y[np.newaxis, :], axis=2)  # Euclidean distance
        matern_kernel = Matern(length_scale=self.length_scale, nu=1.5)  # Matern Kernel (nu=1.5)
        
        # Custom Exponential Decay Function
        custom_term = np.exp(-self.custom_param * dists)
        
        # Combine Matern with Custom Kernel
        K = matern_kernel(X, Y) * custom_term  # Element-wise multiplication

        if eval_gradient:
            # Compute gradient w.r.t. hyperparameters
            matern_grad = matern_kernel(X, Y, eval_gradient=True)[1]  # Get gradient from Matern
            custom_grad = -dists * custom_term[:, :, np.newaxis]  # Derivative w.r.t custom_param
            
            # Stack gradients together
            return K, np.dstack((matern_grad, custom_grad))
        return K

    def diag(self, X):
        """Diagonal elements (self-similarity should be 1)."""
        return np.ones(X.shape[0])

    def is_stationary(self):
        """Returns whether the kernel is stationary."""
        return True

In [ ]:
length_scale = 1.0
nu = 2.5 # i think default is 1.5, but we follow Narayana here who uses 2.5
custom_param = 1.0

matern_kernel = Matern(length_scale= length_scale, nu= nu)
custom_kernel = CustomKernel(length_scale= length_scale, custom_param= custom_param)

combined_kernel = # alpha * (matern_kernel*custom_kernel) + (1-alpha) * matern_kernel + custom_kernel

In [ ]:
kernel = combined_kernel #(length_scale=1.0, custom_param=1.0)  # Initialize Custom Kernel
alpha = 0.1 # noise level, default 1e-10 which assumes y value has no noise
# I've set this to 0.1 but it can go higher for noisier data, also if noise variance is 0.1, try alpha =0.01
optmizer = 'fmin_l_bfgs_b' # default optimizer, can also use 'fmin_l_bfgs_b' or 'fmin_cobyla'

X = samples

gpr = GaussianProcessRegressor(kernel = kernel, alpha = alpha, optimizer = optmizer, random_state = 42)
gpr.fit(X, X)

In [ ]:
# Define custom GPR with Matern kernel
custom_gpr = GaussianProcessRegressor(kernel=Matern(length_scale=1.0, nu=1.5))

# Run Bayesian Optimization with UCB using custom GPR
res = gp_minimize(objective_function, space, n_calls=20, base_estimator=custom_gpr, 
                  acq_func="LCB", acq_func_kwargs={"kappa": 2.0}, random_state=42)